In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pywt
import numpy as np
import pandas as pd
from mne.io import read_raw_fif
import matplotlib
from scipy import signal
import scipy.io
import mne

In [ ]:
def get_avg_psds(freqs,psds):

    FREQ_BANDS = {"delta": [1, 5.0],
                  "theta": [5.0, 8.0],
                  "alpha": [8.0, 13.0],
                  "beta": [13.0, 30.0],
                  "gamma": [30.0, 40.0]}

    psds_bands = []
    for fmin, fmax in FREQ_BANDS.values():
      psds_slice = psds[(freqs >= fmin) & (freqs < fmax)]
      if len(psds_slice) > 0:
          psds_band = psds_slice.mean()
      else:
          psds_band = 1  # or any other value you prefer for empty slices
      psds_bands.append(psds_band)

    psds_bands = np.array(psds_bands)
    relative_psds = psds_bands/psds_bands.sum()     # Relative band values
    abs_psds=psds_bands                 # Absolute band values
#     print('{0:.3f}'.format(relative_psds[0]), ',','{0:.3f}'.format(relative_psds[1]),
#     ',','{0:.3f}'.format(relative_psds[2]), ',', '{0:.3f}'.format(relative_psds[3]),
#           ',','{0:.3f}'.format(relative_psds[4]), ',','{0:.3f}'.format(relative_psds[5]))
    print(relative_psds[0], ',',relative_psds[1], ',',relative_psds[2], ',', relative_psds[3],
          ',',relative_psds[4])
    return relative_psds,abs_psds

In [ ]:
import pandas as pd
import os
import scipy.io

folder_path = '/content/drive/MyDrive/dense/cleaned_data/'

# Load the 8-scale emotion labels file
emotion_labels_path = '/content/drive/MyDrive/dense/cleaned_data_readings.csv'
emotion_labels_data = pd.read_csv(emotion_labels_path)

file_names = []
labels = []

ch_delta_psd = [[] for _ in range(32)]
ch_theta_psd = [[] for _ in range(32)]
ch_alpha_psd = [[] for _ in range(32)]
ch_beta_psd = [[] for _ in range(32)]
ch_gamma_psd = [[] for _ in range(32)]

ch_psd_dfs = []
select_ch = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]
ch_names = ['FP1','AF3','F7','F3','Fz','FC5','FC1','T7','C3','CP5','CP1','P7','P3','PO3','O1','FP2','AF4','F8','F4','FC6','FC2','T8','C4','CP6','CP2','P8','P4','Pz','PO4','O2','Oz']

for i in range(32):
    for j in range(5):
        ch_psd = ch_delta_psd if j == 0 else ch_theta_psd if j == 1 else ch_alpha_psd if j == 2 else ch_beta_psd if j == 3 else ch_gamma_psd
        ch_psd_df = pd.DataFrame({f'ch{i+1}_{["delta", "theta", "alpha", "beta", "gamma"][j]}_psd': ch_psd[i]})
        ch_psd_dfs.append(ch_psd_df)

def append_psd(data, index, fx, preps):
    apsd, abs_apsd = get_avg_psds(fx, preps)
    for i, psd in enumerate(apsd):
        data[i].append(psd)

for index, row in emotion_labels_data.iterrows():
    group = row['Groups']
    file_name = row['file_name']
    emotion_label = row['emotion_category']
    n_fft = 1751  # length of fft
    file_path = os.path.join(folder_path, f"{group}/", f"{file_name}")
    raw = scipy.io.loadmat(file_path)
    print(raw)  # Print the raw data directly
    labels.append(emotion_label)
    file_names.append(file_name)

ch_psd_df = pd.concat(ch_psd_dfs, axis=1)
result_csv_path = '/content/drive/MyDrive/eeg_psd/psd_bandwise_encoded_with_8_scales_labels_and_channels.csv'
ch_psd_df.to_csv(result_csv_path, index=False)

print(f"CSV file created for PSD: {result_csv_path}")
